In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import io
from time import time

In [ ]:
zomatodata = pd.read_csv("./zomato.csv", sep=',', encoding='latin-1')
zomatodata


In [ ]:
#remove data with votes less than 20
zomatodata = zomatodata[~(zomatodata['Votes'] <= 20)] 
#remove duplicate data
zomatodata = zomatodata.drop('Rating text', axis = 1)
zomatodata = zomatodata.drop('Rating color', axis = 1)
zomatodata = pd.get_dummies(zomatodata)
# Display the first 5 rows of the last 12 columns
zomatodata.iloc[:,5:].head(5)

In [ ]:
import numpy as np
# Labels are the values we want to predict
labels = np.array(zomatodata['Aggregate rating'])
# Remove the labels from the features
# axis 1 refers to the columns
zomatodata= zomatodata.drop('Aggregate rating', axis = 1)
# Saving feature names for later use
zomatodatalist = list(zomatodata.columns)
# Convert to numpy array
zomatodata_array = np.array(zomatodata)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(zomatodata, labels, test_size = 0.2, random_state = 0)

In [51]:
print(len(train_features), "train +", len(test_features), "test")

4334 train + 1084 test


In [22]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (4334, 13188)
Training Labels Shape: (4334,)
Testing Features Shape: (1084, 13188)
Testing Labels Shape: (1084,)


In [109]:
time_Start = time()
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 100 decision trees
rf = RandomForestRegressor(n_estimators = 50,n_jobs=-1, random_state=42)
# Train the model on training data
rf.fit(train_features, train_labels);
time_Taken = time() - time_Start
print ("Time Taken after pruning",time_Taken)

Time Taken after pruning 38.89216995239258


In [110]:
predictions = rf.predict(test_features) #predictions on test data
errors = abs(predictions - test_labels)
#print(errors)
print('Mean Absolute Error on test data:', round(np.mean(errors), 2))
#print(test_labels)
mape = 100 * (errors / test_labels)
#print('MAPE', mape)
accuracy = 100 - np.mean(mape)
#print('Prediction', predictions)
print('Accuracy of test data:', (accuracy), '%.')

Mean Absolute Error on test data: 0.26
Accuracy of test data: 92.10092167848997 %.


In [83]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(train_labels, pred_train))
print("RMSE of train data" , rms) 
rmse = sqrt(mean_squared_error(test_labels, predictions))
print("RMSE of test data" , rmse) 

RMSE of train data 0.12752163782118728
RMSE of test data 0.34911470232887987


In [ ]:
imp = pd.DataFrame(rf.feature_importances_)
imp['colname'] = zomatodatalist
imp.columns = ['importance','variable']
imp = imp.sort_values(by = ['importance'], ascending = False)
print (imp.sort_values(by = ['importance']))
imp_plot = imp.iloc[:30,:]
print (imp.sort_values(by = ['importance']))
plt.figure()
# imp[:30].plot(kind='barh',fontsize=10)
imp.importance = imp.importance.astype(float)
imp_plot.plot(x="variable", y="importance", kind="bar", fontsize=10, legend=None)
plt.ylabel("Importance", fontsize=15)
plt.xlabel("Variable", fontsize=15)
# print (imp['importance'][0])
# print (imp['variable'][0])
# print (imp)
zomato_data_copy = zomatodata
print(zomato_data_copy)
for i in range (0,len(imp)):
    if imp['importance'][i] < 0.0015:
        zomato_data_copy = zomato_data_copy.drop(str(imp['variable'][i]), axis = 1)
zomatodatalist = list(zomato_data_copy.columns)
# Convert to numpy array
zomato_data_array = np.array(zomato_data_copy)
print (len(zomatodatalist))
print (zomatodatalist)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(zomatodata_array, labels, test_size = 0.20, random_state = 0)
from sklearn.ensemble import RandomForestRegressor

In [116]:
# Instantiate model with 100 decision trees
time_Start = time()
rf = RandomForestRegressor(n_estimators = 50,n_jobs=-1, random_state=42)
# Train the model on training data
rf.fit(train_features, train_labels);
time_Taken = time() - time_Start
print ("Time Taken after pruning",time_Taken)

Time Taken after pruning 153.90017557144165


In [ ]:

predictions = rf.predict(test_features) #predictions on test data
errors = abs(predictions - test_labels)

In [90]:
print('Mean Absolute Error test data after pruning:', round(np.mean(errors), 2))
mape = 100 * (errors / test_labels)
#print('MAPE', mape)
accuracy = 100 - np.mean(mape)
#print('Prediction', predictions)
print('Accuracy of test data:', round(accuracy, 2), '%.')
from sklearn.metrics import r2_score
test_acc = r2_score(test_labels,predictions)

pred_train = rf_most_important.predict(train_important)
train_acc = r2_score(train_labels,pred_train)
print ("R2 score of train data after pruning", train_acc)
print ("R2 score of test data after pruning", test_acc)

from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(train_labels, pred_train))
print("RMSE of train data after pruning", rms) 
rmse = sqrt(mean_squared_error(test_labels, predictions))
print("RMSE of test data after pruning", rmse) 

Mean Absolute Error test data after pruning: 0.26
Accuracy of test data: 92.1 %.
R2 score of train data after pruning 0.9465696386505652
R2 score of test data after pruning 0.6152869093731188
RMSE of train data after pruning 0.12752163782118728
RMSE of test data after pruning 0.3482920742763034


In [ ]:
print('Parameters currently in use:\n')
print(rf.get_params())

Parameters currently in use:

{'bootstrap': True, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 'warn', 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [50, 155, 261, 366, 472, 577, 683, 788, 894, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores on 21 most important features 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_important, train_labels)
#rf_random.fit(train_features, train_labels)


In [ ]:
rf_random.best_params_


In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 300, random_state = 42)
base_model.fit(train_important, train_labels)
base_accuracy = evaluate(base_model, test_important, test_labels)

In [ ]:
best_random = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth=20, min_samples_leaf=1,min_samples_split=10)
best_random.fit(train_important, train_labels)
base_accuracy = evaluate(base_model, test_important, test_labels)
random_accuracy = evaluate(best_random, test_important, test_labels)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))